https://enrico-alemani.medium.com/flatten-nested-dictionaries-in-pandas-using-glom-7948345c88f5
https://stackoverflow.com/questions/40588852/pandas-read-nested-json
https://stackoverflow.com/questions/35932060/dataframe-of-dataframes-in-python-pandas
https://www.statology.org/pandas-keep-columns/
https://chriswilcox.dev/blog/2020/09/02/Diving_into_GitHub_with_BigQuery_and_Python.html


In [1]:
import pandas as pd
from glom import glom
import json
import numpy as np
import gzip

In [8]:
import logging

In [25]:
f = open('raw_2022-03-25-0.json')

In [26]:
df = pd.read_json(f, lines=True)

In [27]:
df1 = pd.DataFrame(df.iloc[0])

In [28]:
df1

,0
id,20926542665
type,WatchEvent
actor,"{'id': 15946300, 'login': 'TakuyaSama', 'displ..."
repo,"{'id': 158927812, 'name': 'Peltoche/lsd', 'url..."
payload,{'action': 'started'}
public,True
created_at,2022-03-25 00:00:00+00:00
org,NaN


In [29]:
df.isna().any()

id            False
type          False
actor         False
repo          False
payload       False
public        False
created_at    False
org            True
dtype: bool

In [30]:
df.isnull().any()

id            False
type          False
actor         False
repo          False
payload       False
public        False
created_at    False
org            True
dtype: bool

In [1]:
schema = [
    {
        "name": "id",
        "type": "STRING",
        "description": "Unique event ID"
    },
    {
        "name": "type",
        "type": "STRING",
        "description": "https://developer.github.com/v3/activity/events/types/"
    },
    {
        "name":"actor_id",
        "type": "INTEGER",
        "description": "Numeric ID of the GitHub actor"
    },
    {
        "name": "actor_login",
        "type": "STRING",
        "description": "Actor's GitHub login"
    },
    {
        "name": "actor_gravatar_id",
        "type": "STRING",
        "description": "Actor's Gravatar ID"
    },
    {
        "name": "actor_url",
        "type": "STRING",
        "description": "Actor's profile URL"
    },
    {
        "name": "actor_avatar_url",
        "type": "STRING",
        "description": "Actor's Gravatar URL"
    },
    {
        "name": "repo_id",
        "type": "INTEGER",
        "description": "Numeric ID of the GitHub repository"
    },
    {
        "name": "repo_name",
        "type": "STRING",
        "description": "Repository name"
    },
    {
        "name": "repo_url",
        "type": "STRING",
        "description": "Repository URL"
    },
    {
        "name": "public",
        "type": "BOOLEAN",
        "description": "Always true for this dataset since only public activity is recorded."
    },
    {
        "name": "created_at",
        "type": "TIMESTAMP",
        "description": "Timestamp of associated event"
    },
    {
        "name": "org_id",
        "type": "INTEGER",
        "description": "Numeric ID of the GitHub org"
    },
    {
        "name": "org_login",
        "type": "STRING",
        "description": "Org's GitHub login"
    },
    {
        "name": "org_gravatar_id",
        "type": "STRING",
        "description": "Org's Gravatar ID"
    },
    {
        "name": "org_avatar_url",
        "type": "STRING",
        "description": "Org's Gravatar URL"
    },
    {
        "name": "org_url",
        "type": "STRING",
        "description": "Org's profile URL"
    }
]

In [2]:
schema

[{'name': 'id', 'type': 'STRING', 'description': 'Unique event ID'},
 {'name': 'type',
  'type': 'STRING',
  'description': 'https://developer.github.com/v3/activity/events/types/'},
 {'name': 'actor_id',
  'type': 'INTEGER',
  'description': 'Numeric ID of the GitHub actor'},
 {'name': 'actor_login',
  'type': 'STRING',
  'description': "Actor's GitHub login"},
 {'name': 'actor_gravatar_id',
  'type': 'STRING',
  'description': "Actor's Gravatar ID"},
 {'name': 'actor_url', 'type': 'STRING', 'description': "Actor's profile URL"},
 {'name': 'actor_avatar_url',
  'type': 'STRING',
  'description': "Actor's Gravatar URL"},
 {'name': 'repo_id',
  'type': 'INTEGER',
  'description': 'Numeric ID of the GitHub repository'},
 {'name': 'repo_name', 'type': 'STRING', 'description': 'Repository name'},
 {'name': 'repo_url', 'type': 'STRING', 'description': 'Repository URL'},
 {'name': 'public',
  'type': 'BOOLEAN',
  'description': 'Always true for this dataset since only public activity is reco

In [2]:
spec_org = {
    'id':'id',
    'type':'type',
    'actor_id':'actor.id',
    'actor_login':'actor.login',
    'actor_gravatar_id':'actor.gravatar_id',
    'actor_url':'actor.url',
    'actor_avatar_url':'actor.avatar_url',
    'repo_id':'repo.id',
    'repo_name':'repo.name',
    'repo_url':'repo.url',
    #'payload':'payload',
    'public':'public',
    'created_at':'created_at',
    'org_id':'org.id',
    'org_login':'org.login',
    'org_gravatar_id':'org.gravatar_id',
    'org_avatar_url':'org.avatar_url',
    'org_url':'org.url'
}

In [3]:
spec = {
    'id':'id',
    'type':'type',
    'actor_id':'actor.id',
    'actor_login':'actor.login',
    'actor_gravatar_id':'actor.gravatar_id',
    'actor_url':'actor.url',
    'actor_avatar_url':'actor.avatar_url',
    'repo_id':'repo.id',
    'repo_name':'repo.name',
    'repo_url':'repo.url',
    #'payload':'payload',
    'public':'public',
    'created_at':'created_at'
}

In [37]:
df.iloc[5].org

{'id': 66639503,
 'login': 'sepinf-inc',
 'gravatar_id': '',
 'url': 'https://api.github.com/orgs/sepinf-inc',
 'avatar_url': 'https://avatars.githubusercontent.com/u/66639503?'}

In [38]:
df.iloc[5].actor

{'id': 7276994,
 'login': 'lfcnassif',
 'display_login': 'lfcnassif',
 'gravatar_id': '',
 'url': 'https://api.github.com/users/lfcnassif',
 'avatar_url': 'https://avatars.githubusercontent.com/u/7276994?'}

In [39]:
print(glom(df.iloc[5], spec))

{'id': 20926542675, 'type': 'IssueCommentEvent', 'actor_id': 7276994, 'actor_login': 'lfcnassif', 'actor_gravatar_id': '', 'actor_url': 'https://api.github.com/users/lfcnassif', 'actor_avatar_url': 'https://avatars.githubusercontent.com/u/7276994?', 'repo_id': 37318294, 'repo_name': 'sepinf-inc/IPED', 'repo_url': 'https://api.github.com/repos/sepinf-inc/IPED', 'public': True, 'created_at': Timestamp('2022-03-25 00:00:00+0000', tz='UTC')}


In [40]:
temp = glom(df.iloc[5], spec)

In [41]:
if df.iloc[5].org is np.nan:
    print('derp')
else:
    print('dorp')

dorp


In [4]:
data = []
with gzip.open('2022-03-30-23.json.gz', 'rt', encoding='UTF-8') as f:
    for line in f:
        j_content = json.loads(line)
        if j_content.get('org') is None:
            d_line = glom (j_content, spec)
        else:
            d_line = glom(j_content, spec_org)
        data.append(d_line)

In [5]:
dff = pd.DataFrame(data)

In [6]:
dff

,id,type,actor_id,actor_login,actor_gravatar_id,actor_url,actor_avatar_url,repo_id,repo_name,repo_url,public,created_at,org_id,org_login,org_gravatar_id,org_gravatar_url,org_url
0,21029121984,PullRequestEvent,49699333,dependabot[bot],,https://api.github.com/users/dependabot[bot],https://avatars.githubusercontent.com/u/49699333?,166808256,jellyfer/shopify_challenge,https://api.github.com/repos/jellyfer/shopify_...,True,2022-03-30T23:00:00Z,NaN,NaN,NaN,NaN,NaN
1,21029121985,PushEvent,45012333,davidsmatlak,,https://api.github.com/users/davidsmatlak,https://avatars.githubusercontent.com/u/45012333?,392091133,davidsmatlak/azure-rest-api-specs,https://api.github.com/repos/davidsmatlak/azur...,True,2022-03-30T23:00:00Z,NaN,NaN,NaN,NaN,NaN
2,21029121989,CreateEvent,97083797,s0214m,,https://api.github.com/users/s0214m,https://avatars.githubusercontent.com/u/97083797?,476077641,s0214m/wonderful_site,https://api.github.com/repos/s0214m/wonderful_...,True,2022-03-30T23:00:00Z,NaN,NaN,NaN,NaN,NaN
3,21029121990,PushEvent,1118113,NickDMansfield,,https://api.github.com/users/NickDMansfield,https://avatars.githubusercontent.com/u/1118113?,473873885,NickDMansfield/connect4engine,https://api.github.com/repos/NickDMansfield/co...,True,2022-03-30T23:00:00Z,NaN,NaN,NaN,NaN,NaN
4,21029121994,PullRequestReviewEvent,34475974,renovate-approve[bot],,https://api.github.com/users/renovate-approve[...,https://avatars.githubusercontent.com/u/34475974?,357072993,dbsystel/trivy-vulnerability-explorer,https://api.github.com/repos/dbsystel/trivy-vu...,True,2022-03-30T23:00:00Z,20337168.0,dbsystel,,https://avatars.githubusercontent.com/u/20337168?,https://api.github.com/orgs/dbsystel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267568,21030056144,PullRequestReviewCommentEvent,2279037,mcvsubbu,,https://api.github.com/users/mcvsubbu,https://avatars.githubusercontent.com/u/2279037?,19961085,apache/pinot,https://api.github.com/repos/apache/pinot,True,2022-03-30T23:52:52Z,47359.0,apache,,https://avatars.githubusercontent.com/u/47359?,https://api.github.com/orgs/apache
267569,21030056165,PullRequestReviewCommentEvent,2279037,mcvsubbu,,https://api.github.com/users/mcvsubbu,https://avatars.githubusercontent.com/u/2279037?,19961085,apache/pinot,https://api.github.com/repos/apache/pinot,True,2022-03-30T23:56:36Z,47359.0,apache,,https://avatars.githubusercontent.com/u/47359?,https://api.github.com/orgs/apache
267570,21030056124,PullRequestReviewCommentEvent,2279037,mcvsubbu,,https://api.github.com/users/mcvsubbu,https://avatars.githubusercontent.com/u/2279037?,19961085,apache/pinot,https://api.github.com/repos/apache/pinot,True,2022-03-30T23:57:09Z,47359.0,apache,,https://avatars.githubusercontent.com/u/47359?,https://api.github.com/orgs/apache
267571,21030056127,PullRequestReviewCommentEvent,2279037,mcvsubbu,,https://api.github.com/users/mcvsubbu,https://avatars.githubusercontent.com/u/2279037?,19961085,apache/pinot,https://api.github.com/repos/apache/pinot,True,2022-03-30T23:57:24Z,47359.0,apache,,https://avatars.githubusercontent.com/u/47359?,https://api.github.com/orgs/apache


In [7]:
dff.to_parquet('2022-03-30-23.parquet')

In [11]:
def json2parquet(src_file):
    if not src_file.endswith('.json.gz'):
        logging.error("Can only accept source files in json.gz format, for the moment")
        return
    data = []
    with gzip.open(src_file, 'rt', encoding='UTF-8') as f:
        for line in f:
            j_content = json.loads(line)
            if j_content.get('org') is None:
                d_line = glom (j_content, spec)
            else:
                d_line = glom(j_content, spec_org)
            data.append(d_line)
    pd.DataFrame(data).to_parquet(src_file.replace('.json.gz', '.parquet'))

In [12]:
json2parquet('2022-03-30-23.json.gz')